# 1 CARGAR EL MODELO PRE ENTRENADO

In [26]:
from sagemaker import get_execution_role
from sagemaker import Session

role = 'arn:aws:iam::095498312899:role/service-role/AmazonSageMaker-ExecutionRole-20231026T100283'
sess = Session()
bucket = sess.default_bucket()

import tensorflow as tf
import keras
from keras.models import load_model

tf_framework_version = tf.__version__
import h5py
import numpy as np
import os
import tarfile
from sagemaker.tensorflow.serving import TensorFlowModel

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Acer\AppData\Local\sagemaker\sagemaker\config.yaml


In [13]:
input_bucket = "pteriscope"
prefix = "PterygiumDetectionModel"
filename = "ResNext50.zip"

In [17]:
inputs = sess.upload_data(path=filename, key_prefix='model')

In [14]:
!aws s3 cp s3://$input_bucket/ $filename

In [15]:
!aws s3 unzip model.zip -d keras_model


usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help

aws: error: argument subcommand: Invalid choice, valid choices are:

ls                                       | website                                 
cp                                       | mv                                      
rm                                       | sync                                    
mb                                       | rb                                      
presign                                 



In [ ]:
import boto3

s3 = boto3.client('s3')
bucket_name = bucket

# Obtener el modelo JSON desde S3
obj = s3.get_object(Bucket=bucket_name, Key='model/resnext50_model.json')
loaded_model_json = obj['Body'].read().decode('utf-8')
loaded_model = keras.models.model_from_json(loaded_model_json)

# Obtener los pesos del modelo desde S3
s3.download_file(bucket_name, 'model/resnext50_weights_.hdf5', 'local_model.h5')
loaded_model.load_weights('local_model.h5')
loaded_model.summary()

In [18]:
with open(bucket.join('model/resnext50_model.json'), 'r') as fp:
    loaded_model_json = fp.read()
    loaded_model = keras.models.model_from_json(loaded_model_json)
loaded_model.load_weights(bucket.join('model/resnext50_weights_.hdf5'))
loaded_model.summary()

with open(os.path.join('keras_model/model', 'model.json'), 'r') as fp:
    loaded_model_json = fp.read()
    loaded_model = keras.models.model_from_json(loaded_model_json)
loaded_model.load_weights('keras_model/model/model.h5')
loaded_model.summary()

FileNotFoundError: [Errno 2] No such file or directory: 'msagemaker-us-east-1-095498312899osagemaker-us-east-1-095498312899dsagemaker-us-east-1-095498312899esagemaker-us-east-1-095498312899lsagemaker-us-east-1-095498312899/sagemaker-us-east-1-095498312899rsagemaker-us-east-1-095498312899esagemaker-us-east-1-095498312899ssagemaker-us-east-1-095498312899nsagemaker-us-east-1-095498312899esagemaker-us-east-1-095498312899xsagemaker-us-east-1-095498312899tsagemaker-us-east-1-0954983128995sagemaker-us-east-1-0954983128990sagemaker-us-east-1-095498312899_sagemaker-us-east-1-095498312899msagemaker-us-east-1-095498312899osagemaker-us-east-1-095498312899dsagemaker-us-east-1-095498312899esagemaker-us-east-1-095498312899lsagemaker-us-east-1-095498312899.sagemaker-us-east-1-095498312899jsagemaker-us-east-1-095498312899ssagemaker-us-east-1-095498312899osagemaker-us-east-1-095498312899n'

In [21]:
tf.compat.v1.disable_eager_execution()
with open('saved_models/ResNext50/resnext50_model.json', 'r') as json_file:
    json_savedModel= json_file.read()

# Cargamos la arquitectura del modelo
loaded_model = tf.keras.models.model_from_json(json_savedModel)
loaded_model.load_weights('saved_models/ResNext50/resnext50_weights_.hdf5')

Instructions for updating:
Colocations handled automatically by placer.


In [24]:
model_version = '1'
export_dir = 'export/Servo/' + model_version
tf.saved_model.save(loaded_model, export_dir)
model_archive = 'resnext50.tar.gz'
with tarfile.open(model_archive, mode='w:gz') as archive:
    archive.add('export', recursive=True)
    model_data = sess.upload_data(path=model_archive, key_prefix='model')

INFO:tensorflow:Assets written to: export/Servo/1\assets


INFO:tensorflow:Assets written to: export/Servo/1\assets


In [27]:
instance_type = 'ml.t2.medium'
sm_model = TensorFlowModel(model_data=model_data, framework_version=tf_framework_version, role=role)
predictor = sm_model.deploy(initial_instance_count=1, instance_type=instance_type)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Acer\AppData\Local\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Acer\AppData\Local\sagemaker\sagemaker\config.yaml
-----------------------------*

UnexpectedStatusException: Error hosting endpoint tensorflow-inference-2023-10-29-05-22-35-314: Failed. Reason:  Failed to extract model data archive for container "container_1" from URL "s3://sagemaker-us-east-1-095498312899/model/resnext50.tar.gz". Please ensure that the object located at the URL is a valid tar.gz archive..